In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "True"

In [5]:
from omegaconf import OmegaConf as om
from llmfoundry.utils.config_utils import pop_config
from llmfoundry.utils.builders import build_tokenizer
from transformers import AutoTokenizer
from llmfoundry.utils.config_utils import update_batch_size_info

yaml_path = "/home/rshaw/zephyr-training/training/data/yamls/mistral-sft-50sparse-fp16-v0-run1.yaml"
with open(yaml_path) as f:
    cfg = om.load(f)
om.resolve(cfg)

cfg = update_batch_size_info(cfg)

train_loader_config = pop_config(cfg, 'train_loader', must_exist=True)

tokenizer_config = pop_config(cfg, 'tokenizer', must_exist=True)
tokenizer_name = tokenizer_config['name']
tokenizer_kwargs = tokenizer_config.get('kwargs', {})
tokenizer = build_tokenizer(tokenizer_name, tokenizer_kwargs)

chat_tokenizer_name = tokenizer_config.get('chat_template_tokenizer', None)
if chat_tokenizer_name is not None:
    chat_tokenizer = AutoTokenizer.from_pretrained(chat_tokenizer_name)
    tokenizer.chat_template = chat_tokenizer.chat_template

device_train_batch_size = pop_config(cfg, 'device_train_batch_size', must_exist=True)

from scripts.train import build_dataloader

dataloader = build_dataloader(
    train_loader_config,
    tokenizer,
    device_train_batch_size,
)

Map: 100%|██████████| 20000/20000 [00:04<00:00, 4773.04 examples/s]

Sample 398 of the processed training set:

<|system|>
</s>
<|user|>
Write a 10-page historical fiction short story about a group of slaves during the 1800s, focusing on the themes of hope, survival, and resistance. Use descriptive language and vivid imagery to bring the characters and setting to life, while also incorporating accurate historical details and events to enhance the authenticity of the story. Ensure that the story contains a well-developed plot, including a clear conflict and resolution, and that the characters are fully fleshed out with distinct personalities, motivations, and backstories. Lastly, the story should evoke a strong emotional response from the reader, while also providing insight into the experiences of African American slaves during this time period.</s>
<|assistant|>
Title: The Underground Railroad

The sound of crickets chirping fills the night air as a group of slaves huddle together in the darkness, their eyes fixed on the dirt road. They are waiting for

In [6]:
from omegaconf import OmegaConf as om
from llmfoundry.utils.config_utils import pop_config
from llmfoundry.utils.builders import build_tokenizer
from transformers import AutoTokenizer
from llmfoundry.utils.config_utils import update_batch_size_info

yaml_path = "/home/rshaw/zephyr-training/training/data/yamls/run2.yaml"
with open(yaml_path) as f:
    cfg = om.load(f)
om.resolve(cfg)

cfg = update_batch_size_info(cfg)

train_loader_config = pop_config(cfg, 'train_loader', must_exist=True)

tokenizer_config = pop_config(cfg, 'tokenizer', must_exist=True)
tokenizer_name = tokenizer_config['name']
tokenizer_kwargs = tokenizer_config.get('kwargs', {})
tokenizer = build_tokenizer(tokenizer_name, tokenizer_kwargs)

chat_tokenizer_name = tokenizer_config.get('chat_template_tokenizer', None)
if chat_tokenizer_name is not None:
    chat_tokenizer = AutoTokenizer.from_pretrained(chat_tokenizer_name)
    tokenizer.chat_template = chat_tokenizer.chat_template

device_train_batch_size = pop_config(cfg, 'device_train_batch_size', must_exist=True)

from scripts.train import build_dataloader

dataloader2 = build_dataloader(
    train_loader_config,
    tokenizer,
    device_train_batch_size,
)

Map (num_proc=32): 100%|██████████| 10000/10000 [00:35<00:00, 278.64 examples/s]


In [13]:
print(len(dataloader.dataloader))

it1 = iter(dataloader2.dataloader)

40


In [15]:
batch=next(it1)
batch["input_ids"].shape

torch.Size([512, 2048])

In [9]:
print(len(dataloader2.dataloader))

it2 = iter(dataloader2.dataloader)

19


/home/rshaw/zephyr-training/.venv/lib/python3.10/site-packages/trl/trainer/utils.py:147: UserWarning: Could not find response key `[28789, 28766, 489, 11143, 28766, 28767]` in the following instance: <s> <|system|>
You are a friendly chatbot</s> 
<|user|>
Here is a piece of text: Longtime readers of Orlando Sentinel editorials might think they've awakened in the Twilight Zone.
The Sentinel's editorial board has had some epic disagreements with hotelier Harris Rosen. Over a light-rail system between downtown Orlando and International Drive. Over the tax value of his property. Over the plan to tap tourist taxes to build three venues in downtown Orlando. We're on opposite sides again on a current controversy, whether a proposal to legalize resort casinos should allow them in Central Florida.
Yet for the editorial board, these disputes don't overshadow Rosen's achievements as one of Central Florida's most successful and visionary businessmen and one of its most generous philanthropists. He

In [12]:
batch=next(it2)
batch["input_ids"].shape

torch.Size([512, 2048])

In [4]:
model_id = "PY007/TinyLlama-1.1B-Chat-v0.3"

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)

/home/rshaw/zephyr-training/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tok = AutoTokenizer.from_pretrained(
    model_id,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
tok_zephyr = AutoTokenizer.from_pretrained(
    "HuggingFaceH4/zephyr-7b-beta",
)

In [48]:
print(type(tok_zephyr.chat_template))

<class 'str'>


In [63]:
chat_template = """{% for message in messages %}
{% if message['role'] == 'user' %}
{{ '<|im_start|>user' + message['content'] + '<|im_end|>\n' }}
{% elif message['role'] == 'system' %}
{{ '<|im_start|>system\n' + message['content'] + '<|im_end|>\n' }}
{% elif message['role'] == 'assistant' %}
{{ '<|im_start|>assistant\n' + message['content'] + '<|im_end|>\n' }}
{% endif %}
{% if loop.last and add_generation_prompt %}
{{ '<|im_start|>assistant\n' }}
{% endif %}
{% endfor %}
"""

In [64]:
chat_template

"{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|im_start|>user' + message['content'] + '<|im_end|>\n' }}\n{% elif message['role'] == 'system' %}\n{{ '<|im_start|>system\n' + message['content'] + '<|im_end|>\n' }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|im_start|>assistant\n' + message['content'] + '<|im_end|>\n' }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|im_start|>assistant\n' }}\n{% endif %}\n{% endfor %}\n"

In [65]:
tok.chat_template = chat_template

In [18]:
from datasets import load_dataset

dataset_id = "HuggingFaceH4/ultrafeedback_binarized"
dataset = load_dataset(dataset_id, split="train_prefs")
column_names = list(dataset.features)

In [67]:
print(tok.apply_chat_template(dataset[1]["chosen"], tokenize=False))

<|im_start|>userUsing a range of physical activities and creative pursuits, and taking into account factors such as age, gender, and locality, develop a comprehensive list of 25 engaging and nutritious activities specifically designed for children throughout the long, hot months of summer. Consider activities such as swimming, hiking, gardening, cooking classes, arts and crafts, and team sports, and ensure that each suggestion promotes healthy habits and positive attitudes towards exercise and wellbeing. Be sure to provide detailed explanations for each activity, outlining any supplies or props required, necessary supervision, and potential hazards or precautions.<|im_end|>

<|im_start|>assistant
Sure, I'd be happy to help you with that! Here are 25 engaging and nutritious activities specifically designed for children throughout the long, hot months of summer:

1. Swimming: Swimming is a great activity for kids during the summer months. It's a great way to cool off, get some exercise, 

### DPO Dataloader

In [1]:
from datasets import load_dataset

dataset_id = "HuggingFaceH4/ultrafeedback_binarized"
dataset = load_dataset(dataset_id, split="train_prefs")
column_names = list(dataset.features)

/home/rshaw/zephyr-training/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
tokenizer.truncation_side = "left"

In [3]:
import re
def _strip_prefix(s, pattern):
    # Use re.escape to escape any special characters in the pattern
    return re.sub(f"^{re.escape(pattern)}", "", s)

def apply_chat_template(example, tokenizer, assistant_prefix="<|assistant|>\n"):
    if all(k in example.keys() for k in ("chosen", "rejected")):
        
        # TODO: handle case where the dataset has system messages
        assert example["chosen"][0]["role"] == "user"
        assert example["chosen"][0]["role"] == "user"
        
        # Filter out the prompt
        prompt_messages = [[msg for msg in example["chosen"] if msg["role"] == "user"][0]]
        prompt_messages.insert(0, {"role": "system", "content": ""})
        
        chosen_messages = example["chosen"][1:]
        rejected_messages = example["rejected"][1:]
        example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
        example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
        example["text_prompt"] = tokenizer.apply_chat_template(
            prompt_messages, tokenize=False, add_generation_prompt=True
        )

    example["text_chosen"] = _strip_prefix(example["text_chosen"], assistant_prefix)
    example["text_rejected"] = _strip_prefix(example["text_rejected"], assistant_prefix)

    return example

In [ ]:
dataset = dataset.select(range(100000)).map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=column_names,
    desc="Formatting comparisons with prompt template",
)

In [56]:
dataset = dataset.rename_columns(
    {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
)

In [57]:
dataset[0]

{'chosen': "Title: The Noxious Alliance of Social Media and Extremism: A Threat to Human Progress\n\nThe advent of social media platforms has irrevocably revolutionized the way we communicate, share and consume information. While these platforms provide access to information for a global audience and allow individuals to connect regardless of geographical barriers, they have also become a breeding ground for the spread of extremist and violent ideologies, presenting a significant threat to human progress. The pervasiveness of social media, coupled with the anonymity afforded by these platforms, has made it easier for extremists to disseminate their messages, recruit new members, and incite violence against targeted groups, resulting in devastating consequences.\n\nExtremist groups often exploit the viral nature of social media to propagate their ideologies. Fast-paced videos, catchy slogans, and emotionally-charged content are a few of the strategies employed by these groups. Swift and

In [58]:
from trl.trainer.utils import DPODataCollatorWithPadding

data_collator = DPODataCollatorWithPadding(
    tokenizer,
    max_length=2048,
    max_prompt_length=512,
    label_pad_token_id=-100,
    padding_value=0,
    truncation_mode="keep_end",
)

In [111]:
from torch.utils.data import DataLoader

dataloader_params = {
    "batch_size": 2,
    "collate_fn": data_collator,
}

dl = DataLoader(dataset, **dataloader_params)

In [112]:
it = iter(dl)
batch = next(it)

In [113]:
print(batch["chosen_input_ids"].shape)
print(batch["chosen_attention_mask"].shape)



torch.Size([2, 1103])
torch.Size([2, 1103])


In [114]:
print(batch["rejected_input_ids"].shape)
print(batch["rejected_attention_mask"].shape)



torch.Size([2, 1106])
torch.Size([2, 1106])


In [115]:
print(batch["prompt_input_ids"].shape)
print(batch["prompt_attention_mask"].shape)

torch.Size([2, 153])
torch.Size([2, 153])


In [129]:
print(tokenizer.decode(batch["chosen_input_ids"][0,-200:]))

atives: Effective counter-narratives that challenge the extremist narratives should be encouraged and shared on these platforms. This can include sharing positive stories of diversity, resilience, and inclusion, showcasing the beauty of global humanity, and aggressively promoting fact-based content.

4. Collaboration: Governments, civil society organizations, and tech giants should work together to combat this issue. Governments should enact and enforce laws to prevent the promotion of extremist ideologies online, while tech giants need to take investment, to design more effective algorithms that prioritize content promoting healthy social interactions and factual accuracy.

5. Encourage Technology to Fight Extremism: The use of Artificial Intelligence and Machine Learning technologies can be instrumental in detecting and removing harmful content. These systems can analyze patterns in online discourse, flag content worthy of review, and even predict extreme behaviors before they escala

In [128]:
print(tokenizer.decode(batch["rejected_input_ids"][0,-200:]))

can take place in schools, homes, and communities. It is also important to develop tools that can help to identify and remove extremist content from social media platforms. Additionally, it is important to work to build a more inclusive society that does not tolerate hate and discrimination.

The spread of extremist and violent ideologies on social media is a serious problem. However, there are a number of things that can be done to address this problem. By educating people about the dangers of these ideologies, developing tools to remove extremist content, and working to build a more inclusive society, we can help to prevent the spread of these dangerous ideas.</s> 
</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>


In [130]:
print((batch["chosen_attention_mask"][0,-100:]))
print((batch["rejected_attention_mask"][0,-100:]))

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])


In [131]:
print((batch["chosen_labels"][0,-100:]))
print((batch["rejected_labels"][0,-100:]))

tensor([  392,  1872,  8776,  3270, 28725,  1312, 14130,  5220,  1549,   927,
          298,  1388,  8542, 28725,   298,  2621,   680,  5645, 18539,   369,
         4681, 23175,  3036, 21325,  7783,  2809, 14983,   304,  1639,   840,
        13252, 28723,    13,    13, 28782, 28723, 17611,   423,   465, 12511,
          298, 25143, 10145,  1569,  1443, 28747,   415,   938,   302,  3951,
        14773, 23091,   304, 13253, 17504, 14880,   541,   347, 28166,   297,
         6705,   288,   304, 17126, 26299,  3036, 28723,  2957,  4918,   541,
        20765, 11533,   297,  3270,  2312,  8306, 28725,  6673,  3036, 21220,
          302,  4058, 28725,   304,  1019,  6782, 12290, 23961,  1159,   590,
        19491,   380,   778, 14643,  6768, 28723,     2, 28705,    13,     2])
tensor([ 1167,  1872,  8776, 28725, 10423,  7040,   298,  5281,  6215,   392,
         3036, 28725,   304,  2739,   298,  1813,   264,   680, 25573,  6190,
        28725,   478,   541,  1316,   298,  5297,   272,  6049,

In [134]:
print(batch["chosen_input_ids"].shape)
print(batch["rejected_input_ids"].shape)

torch.Size([2, 1103])
torch.Size([2, 1106])


In [135]:
max_length = max(batch["chosen_input_ids"].shape[1], batch["rejected_input_ids"].shape[1])
print(max_length)

1106


In [149]:
from trl.trainer.utils import pad_to_length
import torch

concatenated_batch = {}

for k in batch:
    if k.startswith("chosen") and isinstance(batch[k], torch.Tensor):
        pad_value = -100 if "labels" in k else 0
        concatenated_key = k.replace("chosen", "concatenated")
        concatenated_batch[concatenated_key] = pad_to_length(batch[k], max_length, pad_value=pad_value)

for k in batch:
    if k.startswith("rejected") and isinstance(batch[k], torch.Tensor):
        pad_value = -100 if "labels" in k else 0
        concatenated_key = k.replace("rejected", "concatenated")
        concatenated_batch[concatenated_key] = torch.cat(
            (
                concatenated_batch[concatenated_key],
                pad_to_length(batch[k], max_length, pad_value=pad_value),
            ),
            dim=0,
        )

In [150]:
concatenated_batch["concatenated_input_ids"].shape

torch.Size([4, 1106])

tensor([[  304,  1019,  6782, 12290, 23961,  1159,   590, 19491,   380,   778,
         14643,  6768, 28723,     2, 28705,    13,     2,     0,     0,     0],
        [28723, 27431, 10570,  1074, 28747, 27431,  8275,  1074,   460,   264,
          1598,  1069,     2, 28705,    13,     2,     2,     0,     0,     0],
        [    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2],
        [ 3367,   302,  3618, 28725, 22317, 28725,   304,  1918,  1328, 28725,
         21325,  5277, 17950,   304,  8057,  1162,     2, 28705,    13,     2]])

In [159]:
num = -100

for i in range(concatenated_batch["concatenated_input_ids"].shape[0]):
    print(concatenated_batch["concatenated_input_ids"][i, num:])
    print(concatenated_batch["concatenated_attention_mask"][i,num:])
    print("\n\n")

tensor([ 3270, 28725,  1312, 14130,  5220,  1549,   927,   298,  1388,  8542,
        28725,   298,  2621,   680,  5645, 18539,   369,  4681, 23175,  3036,
        21325,  7783,  2809, 14983,   304,  1639,   840, 13252, 28723,    13,
           13, 28782, 28723, 17611,   423,   465, 12511,   298, 25143, 10145,
         1569,  1443, 28747,   415,   938,   302,  3951, 14773, 23091,   304,
        13253, 17504, 14880,   541,   347, 28166,   297,  6705,   288,   304,
        17126, 26299,  3036, 28723,  2957,  4918,   541, 20765, 11533,   297,
         3270,  2312,  8306, 28725,  6673,  3036, 21220,   302,  4058, 28725,
          304,  1019,  6782, 12290, 23961,  1159,   590, 19491,   380,   778,
        14643,  6768, 28723,     2, 28705,    13,     2,     0,     0,     0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### New Packed Chat Dataloader

In [1]:
!pwd

/home/rshaw/zephyr-training/training/scripts


In [12]:
from omegaconf import OmegaConf as om
from llmfoundry.utils.config_utils import pop_config
from llmfoundry.utils.builders import build_tokenizer
from transformers import AutoTokenizer
from llmfoundry.utils.config_utils import update_batch_size_info

yaml_path = "/home/rshaw/zephyr-training/training/data/yamls/mistral-sft-50sparse-fp16-v0-run1.yaml"
with open(yaml_path) as f:
    cfg = om.load(f)
om.resolve(cfg)

cfg = update_batch_size_info(cfg)

train_loader_config = pop_config(cfg, 'train_loader', must_exist=True)

tokenizer_config = pop_config(cfg, 'tokenizer', must_exist=True)
tokenizer_name = tokenizer_config['name']
tokenizer_kwargs = tokenizer_config.get('kwargs', {})
tokenizer = build_tokenizer(tokenizer_name, tokenizer_kwargs)

chat_tokenizer_name = tokenizer_config.get('chat_template_tokenizer', None)
if chat_tokenizer_name is not None:
    chat_tokenizer = AutoTokenizer.from_pretrained(chat_tokenizer_name)
    tokenizer.chat_template = chat_tokenizer.chat_template

device_train_batch_size = pop_config(cfg, 'device_train_batch_size', must_exist=True)

from scripts.train import build_dataloader

dataloader = build_dataloader(
    train_loader_config,
    tokenizer,
    device_train_batch_size,
)

In [13]:
device_train_batch_size

512

Map: 100%|██████████| 20000/20000 [00:04<00:00, 4957.17 examples/s]

Sample 6411 of the processed training set:

<|system|>
</s>
<|user|>
Write an article on the impact of dietary choices on personal health.</s>
<|assistant|>
Dietary choices play a vital role in personal health. It is no secret that the food we consume has a significant impact on our physical and mental well-being. Choosing nutritious food that is full of essential vitamins and minerals, antioxidants, and fiber can help reduce the risk of chronic disease and improve overall health.

One of the most significant benefits of a healthy diet is preventing chronic diseases such as hypertension, diabetes, and heart disease. For instance, research has shown that reducing the intake of saturated fats, red meat or incorporating fish in the diet can reduce the risk of heart disease. This is because a nutritious diet improves blood pressure, lowers cholesterol levels, and reduces inflammation that triggers chronic diseases.

In addition, eating healthy has a positive impact on mental health. Junk f

In [16]:
len(dataloader.dataloader)

40

In [10]:
20000/1250

16.0

In [5]:
it = iter(dataloader.dataloader)

In [6]:
batch = next(it)

Token indices sequence length is longer than the specified maximum sequence length for this model (2429 > 2048). Running this sequence through the model will result in indexing errors


In [9]:
batch["input_ids"]

tensor([[ 1950, 13305,   745,  ...,  1816,   304,   668],
        [28725,   708,  3209,  ..., 28723,    13,    13],
        [ 9968,  4779,   298,  ...,  3316,   302,  4289],
        ...,
        [28725, 18946,   304,  ..., 17271,   298, 18887],
        [22141, 28723,   415,  ...,   264,   808,   302],
        [19639,   297,  1008,  ...,  2553, 28723,  2909]])

In [11]:
batch["labels"]

tensor([[ 1950, 13305,   745,  ...,  1816,   304,   668],
        [28725,   708,  3209,  ..., 28723,    13,    13],
        [ 9968,  4779,   298,  ...,  3316,   302,  4289],
        ...,
        [28725, 18946,   304,  ..., 17271,   298, 18887],
        [22141, 28723,   415,  ...,   264,   808,   302],
        [19639,   297,  1008,  ...,  2553, 28723,  2909]])

In [17]:
print(tokenizer.batch_decode(batch["input_ids"])[0])

for search engines as a mortgage broker:

1. Research keywords: Use keyword research tools such as Google Keyword Planner or SEMrush to identify relevant keywords that potential clients might use to search for mortgage brokers in your area. Use these keywords in your website's content, headings, and metadata.

2. Meta tags: Ensure that your website's title tags and meta descriptions are optimized for search engines. These tags should be relevant, factual, and attention-grabbing.

3. Valuable content: Create valuable content that is informative and engaging for your readers. Your website should have plenty of informative, well-written content that is optimized for search engines.

4. Local Search: As a mortgage broker, focus on local searches in your area by including your location on your website's metadata and content. 

5. Mobile Friendly: Make sure your website is mobile-friendly by having a responsive design to accommodate mobile and tablet users.

6. Site speed: Optimize your webs

### Datasets

In [1]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")

/home/rshaw/zephyr-training/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/rshaw/zephyr-training/.venv/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [43]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/mistral-7b-sft-beta")

In [3]:
def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})

    example["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return example

train_dataset = dataset.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer})

In [7]:
import random
for index in random.sample(range(len(train_dataset)), 1):
    print(f"Sample {index} of the processed training set:\n\n{train_dataset[index]['text']}")

Sample 174456 of the processed training set:

<|system|>
</s>
<|user|>
Write a biting and humorous satire that lampoons the excessive snobbery and self-importance of the contemporary restaurant scene, poking fun at everything from complicated menus and pretentious food descriptions to the haughty attitudes of chefs and servers. Use exaggerated characters, tongue-in-cheek language, and clever irony to skewer the ridiculousness of an industry that takes itself too seriously and alienates customers with its elitist airs. Focus on witty observations and clever parodies of common restaurant tropes, such as foodie culture, farm-to-table trends, and molecular gastronomy. Above all, aim to entertain and amuse with your clever skewering of an all-too familiar target.</s>
<|assistant|>
As the maitre d' led me to my seat at the latest trendy restaurant, he gave me a disdainful glance that made me feel like a peasant showing up to a royal banquet uninvited. With a menu that looked like a Shakespea

In [33]:
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from trl.trainer.utils import ConstantLengthDataset

packed_dataset = ConstantLengthDataset(
    tokenizer,
    train_dataset,
    dataset_text_field="text",
    formatting_func=None,
    seq_length=2048,
    infinite=False,
    num_of_sequences=1024,
    chars_per_token=3.6,
    eos_token_id=tokenizer.eos_token_id,
)

collator = DataCollatorWithPadding(tokenizer)

dataloader_params = {
    "batch_size": 16,
    "collate_fn": collator,
}

print(isinstance(packed_dataset, torch.utils.data.IterableDataset))

dl = DataLoader(packed_dataset, **dataloader_params)

True


In [34]:
it = iter(dl)

In [35]:
batch = next(it)

In [36]:
batch["input_ids"].shape

torch.Size([16, 2048])

In [37]:
print(tokenizer.batch_decode(batch["input_ids"])[6])

understanding. Utilize sensory exploration, through water play, sandboxes, and manipulatives to keep student's minds strong and engaged.

5. Encourage Questions and Discussion: Encourage children to ask questions and discuss lessons. By asking questions, children can solidify their understanding of the topic and the discussion promotes communication and analysis of new ideas—which is vital to their education.

6. Have a Schedule: Have a basic schedule for each day covering major topics and activities to allow for maximum use of available time. Be flexible, but always have a lesson plan at hand, to maximize time and ensure that your students are mindful.

Remember, lesson plans can change for multiple reasons, but always keep it engaging, interactive, and fun as the intellectual and emotional growth of preschoolers are reliant on them having a truly engaging educational experience.</s> 
<|user|>
Thanks for the tips on how to plan creative lessons and build relationships with parents and

### Old Chat Dataloader

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scripts.chat_dataloader import build_chat_dataloader

/usr/lib/python3/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
from omegaconf import OmegaConf as om
from llmfoundry.utils.config_utils import pop_config
from llmfoundry.utils.builders import build_tokenizer

yaml_path = "/data/yamls/run2.yaml"
with open(yaml_path) as f:
    cfg = om.load(f)
om.resolve(cfg)
                                         
train_loader_config = pop_config(cfg, 'train_loader', must_exist=True)

tokenizer_config = pop_config(cfg, 'tokenizer', must_exist=True)
tokenizer_name = tokenizer_config['name']
tokenizer_kwargs = tokenizer_config.get('kwargs', {})
tokenizer = build_tokenizer(tokenizer_name, tokenizer_kwargs)

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}

In [ ]:
device_train_batch_size = pop_config(cfg, 'device_train_batch_size', must_exist=True)

In [ ]:
%cd scripts

/scripts


In [ ]:
from scripts.train import build_dataloader

dataloader = build_dataloader(
    train_loader_config,
    tokenizer,
    device_train_batch_size,
)

Map (num_proc=32):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(input_ids[idx,-10:])

tensor([21733,   798,  4893,   297, 22303, 12638, 28723,     2, 28705,    13])


In [ ]:
print(labels[idx,-10:])

tensor([21733,   798,  4893,   297, 22303, 12638, 28723,     2, 28705,    13])


In [ ]:
batch = next(iter(dataloader.dataloader))

input_ids = batch["input_ids"]
attention_mask = batch["attention_mask"]
labels = batch["labels"]

num_tokens = 2048
idx = 3

print(input_ids[idx, -num_tokens:])
print(tokenizer.decode(input_ids[idx, -num_tokens:]))
for i in range(1, num_tokens + 1):
    label = labels[idx, -i]
    input_id = input_ids[idx, -i]
    if label == -100:
        print(f"{i}: {input_id} // {label}          <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    else:
        print(f"{i}: {input_id} // {label}")

# print(labels[0, -300:])
# print(input_ids[0, -300:])

tensor([    1,     1,     1,  ...,     2, 28705,    13])
<s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s><s> <|system|>
You are a friendly chatbot</s> 
<|user|>
Examine the various ways in which political patronage has affected the quality and effectiveness of public services, including but not limited to issues related to accountability, efficiency, and democratic governance. Consider the historical and contemporary contexts in which patronage has operated, and assess the potential outcomes of different policy interventions aimed at minimizing its negative effects. Back up your analysis with empirical evidence, case studies, and relevant research literature.</s> 
<|assistant|>
Political patronage is the practice of awarding individuals with

In [ ]:
labels = next(iter(dataloader.dataloader))["labels"]

for row_idx in range(labels.shape[0]):
    print(f"\n---------------------- ROW IDX = {row_idx} ----------------------")
    print(labels[row_idx,-200:])

    # for i in range(20):
    #     print(f"i={i}: {labels[row_idx,i*100:(i+1)*100]}")


---------------------- ROW IDX = 0 ----------------------
tensor([ 9158, 22597,  7649,   852,   298,  9467,  2421, 28725,   304,   736,
          460,  2856,  1581,  4514,   302,  7062, 11715,  3427,   304,  9773,
        28723,  2909,   302,   272,  1080,  1162, 28733,  4717,   281,  2233,
         3024,  5747,  2576,   385, 28725,   446,   282,   314,   270,   753,
          385, 28725,   304,   268,  2262, 10270, 28723,  2957,   281,  2233,
          460,  9566,  7885,   438,  6887, 23127, 28725, 12649, 24363, 28725,
          304,   799,  5903,   697, 28723, 20438,   835,   659,  2856,  9158,
        17937,   369,   460,  4842,   298,   272,  2939, 28725,  2490,   272,
        16720, 28764,   280,  3430, 28725,   690,   349,   264,  1423,   286,
        10623,  3684,   298,   264,  9915, 27545, 28725,   304,   272,   261,
        28764,   423,   293, 28725,   690,   349,   264,  7000,  2751,   302,
          272, 16720, 28764,   280,  3430, 28723,   560,  4518,   298,  7062,
     